In [3]:
import networkx as nx
from collections import deque
import matplotlib.pyplot as plt
import numpy as np
import random
import time


def update_potentials_from_tree_1(graph, tree, root):                        #修改1
    """
    使用广度优先搜索更新生成树中的势能，考虑有向图中的反向边。
    """
        # 初始化势能
    for node in graph.nodes():
        graph.nodes[node]['potential'] = float('inf')
    graph.nodes[root]['potential'] = 0
    
    # 使用队列进行 BFS
    queue = deque([root])
    visited = set([root])

    while queue:
        current = queue.popleft()
        
        # 遍历生成树中当前节点的所有邻居
        for neighbor in tree.neighbors(current):
            if neighbor not in visited:
                edge_cost = graph[current][neighbor]['cost']
                graph.nodes[neighbor]['potential'] = graph.nodes[current]['potential'] - edge_cost
                queue.append(neighbor)
                visited.add(neighbor)
        
        # 遍历生成树中指向当前节点的反向邻居
        for precursor in tree.predecessors(current):
            if precursor not in visited:
                edge_cost = graph[precursor][current]['cost']
                graph.nodes[precursor]['potential'] = graph.nodes[current]['potential'] + edge_cost
                queue.append(precursor)
                visited.add(precursor)

def update_potentials_from_tree(graph, root):
    """
    使用广度优先搜索更新生成树中的势能，考虑有向图中的反向边。
    """
    # 初始化势能
    for node in graph.nodes():
        graph.nodes[node]['potential'] = float('inf')
    graph.nodes[root]['potential'] = 0

    # 使用队列进行 BFS
    queue = deque([root])
    visited = set([root])
    i=0

    while queue:
        current = queue.popleft()
        i+=1
        

        # 遍历当前节点的所有邻居
        for neighbor in graph.neighbors(current):
            if graph[current][neighbor]['in_tree'] and neighbor not in visited:
                edge_cost = graph[current][neighbor]['cost']
                graph.nodes[neighbor]['potential'] = graph.nodes[current]['potential'] - edge_cost
                queue.append(neighbor)
                visited.add(neighbor)
        
        # 遍历所有指向当前节点的反向邻居（即所有以当前为目标的节点）
        for precursor in graph.predecessors(current):
            if graph[precursor][current]['in_tree'] and precursor not in visited:
                edge_cost = graph[precursor][current]['cost']
                graph.nodes[precursor]['potential'] = graph.nodes[current]['potential'] + edge_cost
                queue.append(precursor)
                visited.add(precursor)
                
                
                
def find_min_reduced_cost_arc(graph):
    """
    根据节点势能和边的费用，找到具有最小减少成本的弧。
    该函数假设每个节点在图的节点属性中都有一个 'potential' 属性。
    """
    min_reduced_cost = float('inf')
    min_arc = None

    for u, v in graph.edges():
        # 从图中获取节点 u 和 v 的势能
        potential_u = graph.nodes[u]['potential']
        potential_v = graph.nodes[v]['potential']

        # 计算减少成本
        reduced_cost = graph[u][v]['cost'] - potential_u + potential_v

        # 检查是否这是到目前为止的最小减少成本，并且流量小于容量
        #if reduced_cost < min_reduced_cost and graph[u][v]['flow'] < graph[u][v]['capacity']:
        if reduced_cost < min_reduced_cost:
            min_reduced_cost = reduced_cost
            min_arc = (u, v)
    #print(min_reduced_cost)  
    
    if(min_reduced_cost>=0):
        min_arc = None
    
    return min_arc






def find_cycle(graph,tree, u,v):
    """
    在包含 'in_tree' 属性的生成树中找到添加进入弧后形成的闭环。
    graph: NetworkX 图对象
    entering_arc: 元组 (u, v)，表示新添加的边
    """
    
    
    tree_0=tree.to_undirected()
    # 在生成树中找到从 v 到 u 的唯一路径
    try:
        path = nx.shortest_path(tree_0, source=v, target=u)
    except nx.NetworkXNoPath:
        return None  # 如果没有路径，返回 None
    
    # 将进入弧添加到路径末尾形成闭环
    cycle = path + [v]
    return cycle

def update_flows(graph, cycle, flow_update):
    """
    更新图中回路上的流。
    """
    for i in range(len(cycle) - 1):
        u, v = cycle[i], cycle[i + 1]
        if graph.has_edge(u, v):
            graph[u][v]['flow'] += flow_update
        else:
            graph[v][u]['flow'] -= flow_update

            

            

def update_tree(graph, entering_arc, exit_arc):
    """
    更新生成树，加入进入弧，移除退出弧
    """
    # 加入进入弧到生成树
    if entering_arc:
        u_enter, v_enter = entering_arc
        graph[u_enter][v_enter]['in_tree'] = True
    
    # 移除退出弧出生成树
    if exit_arc:
        u_exit, v_exit = exit_arc
        graph[u_exit][v_exit]['in_tree'] = False    
    
    
            
def network_simplex_method(graph,tree):
    """
    网络单纯形法的主函数。
    graph: networkx的图，其中边的属性包括'cost'(费用), 'capacity'(容量) 和 'flow'(流量)。
    potentials: 节点的初始势能字典。
    """
    
    i=0
    operation_count = 0
    while True:
        
        
        
        operation_count += 1
            
            
        entering_arc = find_min_reduced_cost_arc(graph)
        #if i/1000==i//1000 :
            #print(i)
            #print(entering_arc)
           # break
        #i=i+1
        # 步骤2: 找到进入的弧
        
        
        #print(entering_arc)
        if entering_arc is None:
            #print(i)
            break  # 已找到最优解

        # 步骤3: 找到回路
        cycle = find_cycle(graph,tree, *entering_arc)
        #print(cycle)
        flow_update=float('inf')
        # 步骤4: 计算流量更新量
        for u, v in zip(cycle[:-1], cycle[1:]):
            if graph.has_edge(u, v) and flow_update>graph[u][v]['capacity'] - graph[u][v]['flow']:        
                flow_update =graph[u][v]['capacity'] - graph[u][v]['flow']
                exit_arc=u,v
            if graph.has_edge(v, u) and flow_update>graph[v][u]['flow']:
                flow_update =graph[v][u]['flow']
                exit_arc=v,u
        #print(flow_update)
        #print(exit_arc)                
       
       
        # 步骤5: 更新流量
        update_flows(graph, cycle, flow_update)
        
        if exit_arc!=entering_arc:
            update_tree(graph, entering_arc, exit_arc)
            # 重新计算势能 
            tree.add_edge(*entering_arc)  # 加入新的边
            tree.remove_edge(*exit_arc)
            update_potentials_from_tree_1(graph,tree, 1)
            
                               
        

    # 返回流量最优分配的图和势能
    return graph , tree ,operation_count 


    
    


In [4]:
def calculate_total_cost(graph):
    total_cost = 0
    for u, v, data in graph.edges(data=True):
        total_cost += data['cost'] * data['flow']
        #print(total_cost)
       # print(data['flow'])
    return total_cost


def apply_network_simplex_to_subgraphs(graph,m,n):
    
    global costs_over_iterations
    
    #previous_total_cost = None
    j=0
    tree_edges=set()
    tree_edges = [(x, y) for x, y in graph.edges if graph.edges[x, y].get('in_tree', False)]        #修改2
    tree = nx.DiGraph()
    tree.add_edges_from(tree_edges)
    update_potentials_from_tree_1(graph ,tree,0)
    pivot=0
    count=0
    while True :
        j=j+1
        
        
        sub_graph, count_k =create_subgraph(graph,tree,m,n)
        count += count_k
    
    
        sub_graph,tree, pivot_k=network_simplex_method(sub_graph,tree)  
        pivot += pivot_k
    
    
        
        # 更新原图的流量和势能
        for u, v, data in sub_graph.edges(data=True):
            if graph.has_edge(u, v):
                graph[u][v]['flow'] = data['flow']
                if 'in_tree' in data:
                    graph[u][v]['in_tree'] = data['in_tree']
                    
        
        for i, data in sub_graph.nodes(data=True):
            if graph.has_node(i):
                graph.nodes[i]['potential'] = data.get('potential', graph.nodes[i].get('potential'))
        
        #update_collections(sub_graph)
        if find_min_reduced_cost_arc(graph) == None:
            
            #print(calculate_total_cost(graph))
            break
        #print(calculate_total_cost(graph))  
        #current_total_cost = calculate_total_cost(graph)
        #print(current_total_cost)
        #costs_over_iterations.append(current_total_cost)
    
    
    
        #if previous_total_cost is not None:
            #print(current_total_cost)
            #if current_total_cost  ==  previous_total_cost:
                #print(f"Stopping iteration {j} due to minimal cost change.")
                #break
        
        #previous_total_cost = current_total_cost
        #print(previous_total_cost)
  
    return graph,pivot , count 



In [49]:
# 全局变量
A = []
B = []

def initialize_global_variables(graph):
    global A, B
    A = list(graph.edges()) 
    B = []
    random.shuffle(A)  # 随机排序集合A



def update_collections(sub_graph):
    global A
    # 获取所有边并转换为列表格式
    all_edges = list(sub_graph.edges())
    
    # 将所有边加入 A 的末尾
    A.extend(all_edges)



    
    
def create_subgraph(graph, tree, p, q):
    global A, B
    
    
    basis = set()  # 使用集合避免重复

    # 包括所有 in_tree 为 True 的边
    for u, v in tree.edges():
        basis.add((u, v))
        if (u, v) in A:
            A.remove((u, v))
        
    
    total_edges = len(graph.edges())
    qN = int(total_edges * q)
    pN = int(total_edges * p)
    bN = int(total_edges * q*0.1)
    count_i=0
    while True:
        
       # if A == None : break 
        
        selected_from_A = A[:qN]
        del A[:qN]
        selected_from_B = B[:bN]  
        del B[:bN]

    # 合并选择，并计算相对成本系数
        candidates = selected_from_A + selected_from_B
        #candidates = selected_from_A 
        candidates_sorted = sorted(
        candidates, key=lambda x: graph[x[0]][x[1]]['cost'] - graph.nodes[x[0]]['potential'] + graph.nodes[x[1]]['potential'])
        
        count_i += qN
        
        first_candidate = candidates_sorted[0]
        u_0, v_0 = first_candidate 
        reduced_cost_0 = graph[u_0][v_0]['cost'] - graph.nodes[u_0]['potential'] + graph.nodes[v_0]['potential']
        
        if reduced_cost_0 >= 0:
            #print(reduced_cost_0)
            
            for u, v in candidates_sorted:
                 B.append((u, v))
            
        else : 
            break 
    
            

    

    # 选择相对成本系数最小的pN个
    selected_candidates = candidates_sorted[:pN]
    unselected_candidates = candidates_sorted[pN:]
    
    #unselected_edges = set(candidates)-set(selected_candidates)
    for u, v in unselected_candidates:
        attrs = graph.get_edge_data(u, v)
        reduced_cost = attrs['cost'] - graph.nodes[u]['potential'] + graph.nodes[v]['potential']
        if reduced_cost <= 20:
            A.append((u, v))  # Reduced cost < 0, add back to A
        else:
            B.append((u, v))  # Reduced cost >= 0, add to B

    
   
    


    # 创建子图
    sub_graph = nx.DiGraph()
    
    sub_graph.add_nodes_from(graph.nodes(data=True)) 
    for u, v in basis:
        attrs = graph.get_edge_data(u, v)
        sub_graph.add_edge(u, v, **attrs)
        A.append((u, v)) 
    for u, v in selected_candidates:
        attrs = graph.get_edge_data(u, v)
        sub_graph.add_edge(u, v, **attrs)
        A.append((u, v)) 
        
    #print("Total edges in A and B:", len(A) + len(B))
    #print(count_i)


    return sub_graph,count_i




In [173]:
import numpy as py
import random

p=100
list1 = [random.randint(1, 1000) for _ in range(p)]
list2 = [random.randint(1, 1000) for _ in range(p)]

# 计算两个列表的总和
sum1 = sum(list1)
sum2 = sum(list2)

# 调整其中一个列表的元素，以确保两个列表的总和相等
while sum1 != sum2:
    if sum1 > sum2:
    # 如果list1的总和大于list2的总和，从list1中随机选择一个元素减小
        index = random.randint(0, p-1)
        reduction = random.randint(1, min(list1[index], sum1 - sum2))
        list1[index] -= reduction
        sum1 -= reduction
    else:
        # 如果list2的总和大于list1的总和，从list2中随机选择一个元素减小
        index = random.randint(0, p-1)
        reduction = random.randint(1, min(list2[index], sum2 - sum1))
        list2[index] -= reduction
        sum2 -= reduction

# 生成一个10x10的矩阵，元素范围在1到100之间（可以根据需要进行调整）
matrix = [[random.randint(1, 100) for _ in range(p)] for _ in range(p)]

In [174]:
def northwest_corner_method(supply, demand):
    allocation = np.zeros((len(supply), len(demand)))
    i, j = 0, 0

    while i < len(supply) and j < len(demand):
        alloc = min(supply[i], demand[j])
        allocation[i][j] = alloc
        supply[i] -= alloc
        demand[j] -= alloc

        if supply[i] == 0 and i < len(supply) :
            i += 1
        elif demand[j] == 0 and j < len(demand) :
            j += 1

    return allocation

# 使用西北角法求解
nw_allocation = northwest_corner_method(list1.copy(), list2.copy())
#print(nw_allocation)



In [165]:

G = nx.DiGraph()
# 添加供应节点和需求节点

for i in range(p):
    G.add_node(i, potential=float('inf'))  
    

for j in range(p):
    G.add_node(p+j, potential=float('inf'))

count=0
# 添加边及其成本
for i, row in enumerate(matrix):
    for j, weight in enumerate(row):
        if nw_allocation[i][j] > 0 :
            G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=True)
            #tree_graph.add_edge(u, v, **data)
            count+=1
        else :
            G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=False)






In [170]:
start_time = time.time()

costs_over_iterations = []
initialize_global_variables(G)
graph ,pivot ,count= apply_network_simplex_to_subgraphs(G , 0.1, 0.2)
end_time = time.time()
print(pivot) 
print(count)

1512
100000


In [146]:
print(end_time-start_time)

16.917156219482422


In [176]:
import time
import pandas as pd

# 假设你有一个算法函数，例如：
def algorithm(param1, param2):
    # 模拟算法运行
    G = nx.DiGraph()
# 添加供应节点和需求节点

    for i in range(p):
        G.add_node(i, potential=float('inf'))  
     

    for j in range(p):
        G.add_node(p+j, potential=float('inf'))

    count=0
# 添加边及其成本
    for i, row in enumerate(matrix):
        for j, weight in enumerate(row):
            if nw_allocation[i][j] > 0 :
                G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=True)
            #tree_graph.add_edge(u, v, **data)
                count+=1
            else :
                G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=False)

    costs_over_iterations = []
    initialize_global_variables(G)
#node_info = G.nodes[980]
#print("Node 1's Information:", node_info)
    optimized_graph, pivot ,count= apply_network_simplex_to_subgraphs(G,param1, param2)
    return pivot ,count

# 定义一个函数来记录运行时间
def record_time(param1, param2):
    start_time = time.time()
    pivot ,count = algorithm(param1, param2)
    end_time = time.time()
    run_time = end_time - start_time
    return run_time,pivot ,count

def record_time2():
    G = nx.DiGraph()
# 添加供应节点和需求节点

    for i in range(p):
        G.add_node(i, potential=float('inf'))  
     

    for j in range(p):
        G.add_node(p+j, potential=float('inf'))

    count=0
# 添加边及其成本
    for i, row in enumerate(matrix):
        for j, weight in enumerate(row):
            if nw_allocation[i][j] > 0 :
                G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=True)
            #tree_graph.add_edge(u, v, **data)
                count+=1
            else :
                G.add_edge(i, p + j, cost=weight, capacity=min(list1[i], list2[j]),flow=nw_allocation[i][j],in_tree=False)

    costs_over_iterations = []
    initialize_global_variables(G)
    
    previous_total_cost = None
    j=0
    tree_edges=set()
    tree_edges = [(x, y) for x, y in G.edges if G.edges[x, y].get('in_tree', False)]        #修改2
    tree = nx.DiGraph()
    tree.add_edges_from(tree_edges)
    update_potentials_from_tree_1(G ,tree,0)
    
    start_time = time.time()
    result, t = network_simplex_method(G,tree)
    end_time = time.time()
    run_time = end_time - start_time
    return run_time



# 定义参数范围
#param1_range = [0.01,0.02,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
#param2_range = [0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
param1_range = [0.1]
param2_range = [0.12,0.14,0.16,0.18,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]
# 创建一个空的DataFrame来保存结果
results = pd.DataFrame(columns=['param1', 'param2', 'run_time'])
#results = pd.DataFrame(columns=['run_time1', 'run_time2'])

# 循环改变参数并记录运行时间
for param1 in param1_range:
    # 动态过滤 param2，使其大于 param1
    filtered_param2_range = [p for p in param2_range if p > param1]
    for param2 in filtered_param2_range:
        # 假设 record_time 是你的运行时间记录函数
        run_time1, pivot, count = record_time(param1, param2)
        # 将结果添加到 DataFrame
        results = results.append({'param1': param1, 'param2': param2, 'pivot':pivot , 'count':count ,'run_time': run_time1}, ignore_index=True)
print(results)
# 将结果保存到CSV文件
results.to_csv('pivot_count_7.csv', index=False)

#print("实验结果已保存到 algorithm_run_times.csv 文件中")


C:\Users\lilingrui\AppData\Local\Temp\ipykernel_23284\3957020149.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'param1': param1, 'param2': param2, 'pivot':pivot , 'count':count ,'run_time': run_time1}, ignore_index=True)
C:\Users\lilingrui\AppData\Local\Temp\ipykernel_23284\3957020149.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'param1': param1, 'param2': param2, 'pivot':pivot , 'count':count ,'run_time': run_time1}, ignore_index=True)
C:\Users\lilingrui\AppData\Local\Temp\ipykernel_23284\3957020149.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'param1': param1, 'param2': param2, 'pivot':pivot , 'count':count ,'run_time':

    param1  param2   run_time     count   pivot
0      0.1    0.12   9.776220   36000.0  1777.0
1      0.1    0.14   7.193199   22400.0  1470.0
2      0.1    0.16   7.180396   22400.0  1551.0
3      0.1    0.18   6.671119   18000.0  1442.0
4      0.1    0.20   6.517733   16000.0  1351.0
5      0.1    0.30   7.577452   66000.0  1458.0
6      0.1    0.40  10.815931  388000.0  2102.0
7      0.1    0.50   9.957412  195000.0  1967.0
8      0.1    0.60   8.777687  114000.0  1862.0
9      0.1    0.70   7.936557  112000.0  1612.0
10     0.1    0.80   7.542601  120000.0  1513.0
11     0.1    0.90   7.807217  216000.0  1613.0
12     0.1    1.00   6.981718  120000.0  1509.0


C:\Users\lilingrui\AppData\Local\Temp\ipykernel_23284\3957020149.py:101: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'param1': param1, 'param2': param2, 'pivot':pivot , 'count':count ,'run_time': run_time1}, ignore_index=True)


In [48]:
print(results)
results.to_csv('algorithm_run_times7.csv', index=False)

   run_time1  run_time2
0   1.414946  13.720683
1   1.659686  13.870546
2   1.578667  15.560563
3   1.499075  14.772108
4   1.493313  13.713493
